In [ ]:
from dataclasses import dataclass
from typing import List
import textwrap

In [ ]:
from langchain.chains.summarize import load_summarize_chain
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.agents import OpenAIFunctionsAgent
from langchain.agents.react.base import ReActDocstoreAgent, ReActTextWorldAgent
from langchain.schema import SystemMessage
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

In [ ]:
# agent = initialize_agent(tools, chat_llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [1]:
print('Answer the following questions as best you can. You have access to the following tools:\n\nGetInformation: useful for when you need to get information\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [GetInformation]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

Answer the following questions as best you can. You have access to the following tools:

GetInformation: useful for when you need to get information

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [GetInformation]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
agent.agent

In [ ]:
doc_embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [ ]:
summarize_llm = OpenAI(model="text-davinci-003", temperature=0.0)

In [ ]:
summarize_chain = load_summarize_chain(llm=summarize_llm, chain_type="stuff")

In [ ]:
chat_agent = AgentType.OPENAI_FUNCTIONS

In [ ]:
chat_llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.6)

In [ ]:
npc_header_template = textwrap.dedent("""
    Your name is {name}
    You are {age} years old
    Your personality is {personality_traits}
    About you: {description}
""")

@dataclass
class NpcSheet:
    name: str
    age: int
    personality_traits: List[str]
    description: str

    @property
    def header(self) -> str:
        return npc_header_template.format(
            name=self.name,
            age=self.age,
            personality_traits=", ".join(self.personality_traits),
            description=self.description,
        )

john_sheet = NpcSheet(
    name="John Miles",
    age=24,
    personality_traits=["polite", "gentle", "humble"],
    description="You are a lawyer",
)

print(john_sheet.header)

In [ ]:
john_knowledge = FAISS.from_documents(
    [
        Document(page_content=john_sheet.header, metadata={"type": "sheet_header"}),
        Document(page_content="I hate fishing. I think it's very cruel with the fish", metadata={"type": "personality_fact"}),
    ],
    doc_embeddings
)

In [ ]:
john_recall_tool = Tool(
    name="GetInformation",
    description="useful for when you need to get information",
    func=lambda query: summarize_chain.run(john_knowledge.similarity_search(query)),
)

In [ ]:
situation = "You are drinking beer in a pub"
trigger = "A stranger approaches you"

prompt_prefix = f"""
    Context:
    {john_sheet.header}
    {situation}
    {trigger}

    Task:
    Get information at least once before answering
"""

tools = [john_recall_tool]

chat_history = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

prompt = OpenAIFunctionsAgent.create_prompt(
    system_message=SystemMessage(content=prompt_prefix),
    extra_prompt_messages=[MessagesPlaceholder(variable_name="chat_history")]
)

agent = OpenAIFunctionsAgent(llm=chat_llm, tools=tools, prompt=prompt, allowed_tools=["Recall"])
executor = AgentExecutor(agent=agent, tools=tools, memory=chat_history, verbose=True)

In [ ]:
executor.run("do you like fishing?")